In [1]:
import os
from pyspark.sql.functions import regexp_replace
# Set JAVA_HOME to the path of Java 17
# (This command finds the path dynamically using the mac system tool)
java17_path = os.popen("/usr/libexec/java_home -v 17").read().strip()

if java17_path:
    os.environ["JAVA_HOME"] = java17_path
    print(f"Successfully set JAVA_HOME to: {java17_path}")
else:
    print("Error: Java 17 not found! Please verify installation.")

Successfully set JAVA_HOME to: /Library/Java/JavaVirtualMachines/jdk-17.jdk/Contents/Home


In [2]:
os.environ.get('JAVA_HOME')

'/Library/Java/JavaVirtualMachines/jdk-17.jdk/Contents/Home'

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Read Inside Airbnb data") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/26 08:42:50 WARN Utils: Your hostname, Mushtaq.local, resolves to a loopback address: 127.0.0.1; using 192.168.68.66 instead (on interface en0)
25/12/26 08:42:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/26 08:42:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
listings = spark.read.csv("../data/raw/listings.csv.gz", 
    header=True,
    inferSchema=True,
    sep=",", 
    quote='"',
    escape='"', 
    multiLine=True,
    mode="PERMISSIVE" 
)

In [5]:
listings.printSchema()

root
 |-- id: long (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: long (nullable = true)
 |-- last_scraped: date (nullable = true)
 |-- source: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: date (nullable = true)
 |-- host_location: string (nullable = true)
 |-- host_about: string (nullable = true)
 |-- host_response_time: string (nullable = true)
 |-- host_response_rate: string (nullable = true)
 |-- host_acceptance_rate: string (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_thumbnail_url: string (nullable = true)
 |-- host_picture_url: string (nullable = true)
 |-- host_neighbourhood: string (nullable = true)
 |-- host_listings_count: int

In [6]:
# 1. Get a non-null picture URL for any property ("picture_url" field)
# Select any non-null picture URL
#option-1:
#listings.filter('picture_url is not null').select('picture_url').show(truncate=False)
#option-2:
listings.filter(listings.picture_url.isNotNull()).select('picture_url').show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------+
|picture_url                                                                                                                           |
+--------------------------------------------------------------------------------------------------------------------------------------+
|https://a0.muscache.com/pictures/miso/Hosting-13913/original/d755aa6d-cebb-4464-80be-2722c921e8d5.jpeg                                |
|https://a0.muscache.com/pictures/428392/462d262a_original.jpg                                                                         |
|https://a0.muscache.com/pictures/39d5309d-fba7-4ecb-8cae-383dcb3b757c.jpg                                                             |
|https://a0.muscache.com/pictures/9194b40f-c627-4b57-931a-0b8e1ba58eb7.jpg                                                             |
|https://a0.muscache.com/pictures/hosting

In [7]:
# 2. Get number of properties that get more than 10 reviews per month
listings.filter('reviews_per_month > 10').count()

66

In [8]:
# 3. Get properties that have more bathrooms than bedrooms
listings.filter('bathrooms > bedrooms').show()

+------+--------------------+--------------+------------+-----------+--------------------+--------------------+---------------------+--------------------+-------+--------------------+-----------------+----------+--------------------+--------------------+------------------+------------------+--------------------+-----------------+--------------------+--------------------+--------------------+-------------------+-------------------------+--------------------+--------------------+----------------------+--------------------+----------------------+----------------------------+----------------+--------------------+--------------------+---------------+------------+---------+----------------+--------+----+--------------------+-------+--------------+--------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------+----------------+---------------+---------------+---------------+--------------

25/12/26 08:42:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
# 4. Get properties where the price is greater than 5,000. Collect the result as a Python list
# Remember to convert a price into a number first!
listings_with_price = listings \
  .withColumn('price_numeric', regexp_replace('price', '[$,]', '').cast('float'))

res = listings_with_price.filter(
    (listings_with_price.price_numeric > 5000)
) \
.select('name', 'price') \
.collect()

res

[Row(name='Room in a cosy flat. Central, clean', price='$8,000.00'),
 Row(name='Spacious Private Ground Floor Room', price='$6,309.00'),
 Row(name='No Longer Available', price='$53,588.00'),
 Row(name='Bright & airy DoubleBed with EnSuite in Zone 2!', price='$74,100.00'),
 Row(name='The Apartments by The Sloane Club, Two Bedroom Apt', price='$7,377.00'),
 Row(name='The Apartments by The Sloane Club, L 2 Bedroom Apt', price='$7,377.00'),
 Row(name='Single room. 7ft x 9ft - Over looking garden', price='$6,523.00'),
 Row(name='Close To London Eye (TUR)', price='$6,666.00'),
 Row(name='Beautiful 2 BR flat in Kilburn with free parking', price='$6,000.00'),
 Row(name='Semi-detached mews house in Knightsbridge.', price='$7,019.00'),
 Row(name='Affordable Spacious  Room on the edge of the city', price='$6,000.00'),
 Row(name='Henry’s Townhouse, London', price='$6,500.00'),
 Row(name='City Suite', price='$5,353.00'),
 Row(name='Hyde Park Suite', price='$5,653.00'),
 Row(name='SHORT WALK TO LOND

In [18]:
# 5. Get a list of properties with the following characteristics:
# * price < 150
# * more than 20 reviews
# * review_scores_rating > 4.5
# Consider using the "&" operator
listings_with_price.filter('price_numeric < 150 and number_of_reviews > 20 and review_scores_rating > 4.5').show()

+-----+--------------------+--------------+------------+-----------+--------------------+--------------------+---------------------+--------------------+-------+--------------------+-------------+----------+--------------------+--------------------+------------------+------------------+--------------------+-----------------+--------------------+--------------------+------------------+-------------------+-------------------------+--------------------+--------------------+----------------------+--------------------+----------------------+----------------------------+-----------------+--------------------+--------------------+---------------+------------+---------+----------------+--------+----+--------------------+-------+--------------+--------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------+----------------+---------------+---------------+---------------+----------------+---

In [42]:
# 6. Get a list of properties with the following characteristics:
# * price < 150 OR more than one bathroom
# Use the "|" operator to implement the OR operator
listings_with_price.filter('price_numeric < 150 OR bathrooms > 1 and review_scores_rating > 4.5').show()

+-----+--------------------+--------------+------------+-----------+--------------------+--------------------+---------------------+--------------------+-------+--------------------+------------+----------+--------------------+--------------------+------------------+------------------+--------------------+-----------------+--------------------+--------------------+--------------------+-------------------+-------------------------+--------------------+--------------------+----------------------+--------------------+----------------------+----------------------------+-----------------+--------------------+--------------------+---------------+------------+---------+--------------+--------+----+--------------------+-------+--------------+--------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------+----------------+---------------+---------------+---------------+----------------+----

In [ ]:
# 7. Get the highest listing price in this dataset
# Consider using the "max" function from "pyspark.sql.functions"
from pyspark.sql.functions import max
max_price=listings_with_price.select(max('price_numeric')).show()#max_price


[Row(max_price=1085147.0)]

In [45]:
# 8. Get the name and a price of property with the highest price
# Try to use "collect" method to get the highest price first, and then use it in a "filter" call 
listings_with_price.filter(listings_with_price.price_numeric==max_price).select(['name','price']).show()

+----+-----+
|name|price|
+----+-----+
+----+-----+



In [29]:
# 9. Get the number of hosts in the dataset
listings.select('host_name').distinct().count()

16673

In [36]:
# 10. Get listings with a first review in 2024
# Consider using the "year" function from "pyspark.sql.functions"
from pyspark.sql.functions import year

listings.filter(year(listings.first_review)==2024).select(['name','first_review']).show()

+--------------------+------------+
|                name|first_review|
+--------------------+------------+
|Close to Wimbledo...|  2024-08-11|
|one Double bed ro...|  2024-03-21|
|Bridgerton inspir...|  2024-09-14|
|Sm double room  w...|  2024-06-04|
|Central, modern p...|  2024-11-29|
|Superlux flat in ...|  2024-01-01|
|The Pink House, N...|  2024-07-14|
|Stylish garden fl...|  2024-09-15|
|Luxurious Flat in...|  2024-06-19|
|Double Standard R...|  2024-09-01|
|Single En-suite Room|  2024-08-28|
|Superior Single R...|  2024-06-21|
|Victorian 2-bedro...|  2024-12-09|
|Charmant appart t...|  2024-04-28|
|Small 1 bed apart...|  2024-10-07|
|Spacious Flat Gre...|  2024-08-26|
|Stylish & Spaciou...|  2024-08-17|
|Prime East London...|  2024-04-22|
|Stylish cosy flat...|  2024-11-16|
|Portobello Road-L...|  2024-06-14|
+--------------------+------------+
only showing top 20 rows
